In [1]:
!pip install pathway panel bokeh --quiet

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.6/77.6 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.6/777.6 kB 48.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.5/26.5 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.3/135.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.6/244.6 kB 17.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 319.1/319.1 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 985.8/985.8 kB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import pandas as pd

df = pd.read_csv('/content/dataset.csv')

# Combine date and time
df['Timestamp'] = pd.to_datetime(df['LastUpdatedDate'] + ' ' + df['LastUpdatedTime'], format='%d-%m-%Y %H:%M:%S')

# Save trimmed CSV for Pathway
df[['Timestamp', 'Occupancy', 'Capacity', 'SystemCodeNumber', 'TrafficConditionNearby', 'QueueLength', 'IsSpecialDay']].to_csv('parking_stream_model2.csv', index=False)

In [3]:
import pathway as pw

class ParkingLot(pw.Schema):
    Timestamp: str
    Occupancy: int
    Capacity: int
    SystemCodeNumber: str
    TrafficConditionNearby: str
    QueueLength: int
    IsSpecialDay: int

data = pw.demo.replay_csv('parking_stream_model2.csv', schema=ParkingLot, input_rate=100)

In [4]:
from datetime import datetime
fmt = "%Y-%m-%d %H:%M:%S"

data_with_time = data.with_columns(
    t = data.Timestamp.dt.strptime(fmt),
    day = data.Timestamp.dt.strptime(fmt).dt.strftime("%Y-%m-%dT00:00:00")
)

In [5]:
import datetime
delta_window = (
    data_with_time.windowby(
        pw.this.t,
        instance=pw.this.SystemCodeNumber,
        window=pw.temporal.tumbling(datetime.timedelta(days=1)),
        behavior=pw.temporal.exactly_once_behavior()
    )
    .reduce(
        t=pw.this._pw_window_end,
        SystemCodeNumber = pw.reducers.any(pw.this.SystemCodeNumber),
        occ=pw.reducers.sum(pw.this.Occupancy),
        cap=pw.reducers.sum(pw.this.Capacity),
        qlen=pw.reducers.sum(pw.this.QueueLength),
        traffic=pw.reducers.max(pw.this.TrafficConditionNearby),
        spday=pw.reducers.max(pw.this.IsSpecialDay),
    )
)

In [6]:
BASE_PRICE = 10
BETA_OCC = 6
BETA_QLEN = 2
BETA_TRAFFIC = 3
BETA_SPECIAL = 1.5

@pw.udf
def advanced_continuous_price(occ, cap, qlen, traffic, spday):
    # Scale occupancy
    occ_ratio = occ / cap if cap else 0

    # Scale queue (cap at 10)
    queue_scaled = min(qlen / 10, 1.0)

    # Convert traffic condition
    traffic_map = {"low": 0, "medium": 0.5, "high": 1.0}
    traffic_val = traffic_map.get(traffic.lower(), 0)

    # Final price with weighted components
    raw_price = BASE_PRICE + (
        BETA_OCC * occ_ratio +
        BETA_QLEN * queue_scaled +
        BETA_TRAFFIC * traffic_val +
        BETA_SPECIAL * spday
    )

    # Round to 1 decimal place
    return round(raw_price, 1)

result = delta_window.with_columns(
    price=advanced_continuous_price(
        pw.this.occ,
        pw.this.cap,
        pw.this.qlen,
        pw.this.traffic,
        pw.this.spday
    )
)

In [7]:
pw.io.csv.write(
    table=result.select(
        # SystemCodeNumber=result.SystemCodeNumber,
        t=result.t,
        price=result.price
    ),
    filename="/content/model2_prices.csv"
)

/usr/local/lib/python3.11/dist-packages/beartype/_util/hint/pep/utilpeptest.py:311: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.Iterable[pathway.internals.expression.ColumnReference] deprecated by PEP 585. This hint is scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". For further commentary and alternatives, see also:
    https://beartype.readthedocs.io/en/latest/api_roar/#pep-585-deprecations
  warn(


In [8]:
import panel as pn
from bokeh.plotting import figure

pn.extension()

lot_id = df['SystemCodeNumber'].unique()[0]

def plot_model2(source):
    fig = figure(title=f"Model 2: Dynamic Price for Lot {lot_id}", x_axis_type="datetime", width=800, height=400)
    fig.line("t", "price", source=source, line_width=2, color="green")
    fig.circle("t", "price", source=source, size=5, color="blue")
    return fig

viz = result.plot(plot_model2, sorting_col="t")
pn.Column(viz).servable()

Column
    [0] Column
        [0] Row
            [0] Markdown(str)
            [1] TooltipIcon(value='Table depends o...)
        [1] Bokeh(figure)

In [11]:
%%capture --no-display
pw.run()

Output()

KeyboardInterrupt: 